# Neural Network Model


In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import pandas as pd

# Import and read the csv.
df = pd.read_csv("../ML_Data_&_Preprocessing/b3_df_nonEncoded.csv", index_col=[0])
df.head()



In [ ]:
df['leads'].value_counts()

In [ ]:
# check on different state values for testing 
df['state'].value_counts()

In [ ]:
df2 = df.loc[df['state']== 'MN']
df2

In [ ]:
df2.info()

In [ ]:
df3 = df2.drop(df2.columns[[0,6]], axis=1)

In [ ]:
df3.info()

# Split into train and test features

In [ ]:
# Split our preprocessed data into our features and target arrays
y = df3['leads'].values
X = df3.drop(['leads'], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Compile, Train, Evaluate our Model 1 - NN

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# using multi-layer perceptron (two layers)
numInputFeatures = len(X_train[0])

# I have X amount of columns
# layer1 = input layer, typically equals number of input variables in data
layer1 = 30
# layer 2 = hidden layer, typically 2/3 of input layer
layer2 = 15
# layer 3 = hidden layer
layer3= 2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=layer1, 
                          input_dim=numInputFeatures, 
                          activation="hard_sigmoid")
)
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer2, 
                             activation="elu"))

# adding a third layer to increase accuracy 
nn.add(tf.keras.layers.Dense(units=layer3, 
                             activation='elu'))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="hard_sigmoid"))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(
    # checkpoint directory and file structure defined above
    filepath=checkpoint_path,
    # notified when checkpoint is being saved to the directory
    verbose=1,
    # checkpoint files take small space
    save_weights_only=True,
    # checkpoints saved every epoch
    save_freq='epoch')

In [ ]:
# Train the model
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100,callbacks=[cp_callback])

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# when brand 1, all states, accuracy = 36%
# when brand 2, all states, accuracy = 36%
# when brand 3, all states, accuracy = 36%

# B1, CA = 78%
# B2, CA = 80%
# B3, CA = 77%

# B3, PA = 100%
# B3, TX = 76%



### B3 Results

In [ ]:
data_results_b3 = {
        'State': ['PA', 'CA', 'TX', 'VA', 'NY', 'NC', 'SD', 'IL', 'DC', 'MA', 'TN', 'MN'],
        'Predictive Accuracy': [1.0, .77, .76, .79, .84, .83, .95, .81, .84, .85, .75, .78],
        'Tier': ['T1', 'T4', 'T4', 'T4', 'T3', 'T3', 'T1', 'T2', 'T1', 'T2', 'T3', 'T2']}
data_results_b3

In [ ]:
# create the new df to display brand, state, and model accuracy 
df_results_b3 = pd.DataFrame(data_results_b3)
df_results_b3

In [ ]:
df_results_b3.to_csv('b3_results.csv')

In [ ]:
# # Export the model to HDF5 file
# nn.save("AlphabetSoupCharity_optimization.h5")